# General Slides

In [1]:
#import plotly.graph_objects as go
#import chart_studio
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np

# Import pptx libraries
import pptx
from pptx import Presentation
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches

from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_THEME_COLOR
from pptx.util import Pt

from pptx.enum.text import PP_ALIGN

import pptx


# Display max columns when viewing data
pd.set_option('display.max_columns', 500)

In [2]:
msft = pd.read_excel('/Users/brandongoldney/Documents/Projects/Python_PPTX/MSFT_Financials.xls', index_col=False)

# Remove any unnamed columns
cols = [string for string in msft.columns if string.lower()[:7] != 'unnamed']
msft = msft[cols]

# Set rows as the index
msft = msft.set_index(['Account'])

#Convert any columns inported as "object" to "float32"
cols = msft.select_dtypes(exclude=['float']).columns
msft[cols] = msft[cols].apply(pd.to_numeric, downcast='float', errors='coerce')

# Drop any columns with nan values
msft = msft.dropna(axis=1, how = 'any')

msft.head()


,Q1 15,Q2 15,Q3 15,Q4 15,Q1 16,Q2 16,Q3 16,Q4 16,Q1 17,Q2 17,Q3 17,Q4 17,Q1 18,Q2 18,Q3 18,Q4 18,Q1 19,Q2 19,Q3 19
Account,,,,,,,,,,,,,,,,,,,
Total Revenue,21729.00,22180.0,20379.00,23796.00,20531.00,20614.0,21928.00,25826.0,23212.0,25605.00,24538.00,28918.00,26819.00,30085.00,29084.00,32471.00,30571.00,33717.0,33055.00
Gross Profit,14568.00,14712.0,13172.00,13924.00,12809.00,12635.0,14084.00,15925.0,15152.0,17560.00,16260.00,17854.00,17550.00,20343.00,19179.00,20048.00,20401.00,23305.0,22649.00
Operating Income,6784.00,6385.0,5793.00,6026.00,5283.00,4190.0,6715.00,7905.0,6723.0,10279.00,7708.00,8679.00,8292.00,10379.00,9955.00,10258.00,10341.00,12405.0,12686.00
Net Income,4985.00,-3195.0,4902.00,5018.00,3756.00,3122.0,5667.00,6267.0,5486.0,8069.00,6576.00,-6302.00,7424.00,8873.00,8824.00,8420.00,8809.00,13187.0,10678.00
EPS,0.61,-0.4,0.61,0.62,0.47,0.4,0.72,0.8,0.7,1.03,0.84,-0.82,0.95,1.14,1.14,1.08,1.14,1.7,1.38


# Load Presentation as Object

In [3]:
# This loads the MASTER presentation
prs = Presentation('General_Slides - MASTER.pptx')

In [4]:
# The code below shows the idx values for each placeholder
slide = prs.slides[1]

for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

0, TITLE (1)
4, SLIDE_NUMBER (13)
15, CHART (8)


**Slide 1: Leverage**

In [5]:
# The code below is needed to access the chart on the second slide (i.e. prs.slides[1])
slide = prs.slides[1] 
chart = slide.shapes[2].chart # Not sure why choosing shapes[1] works but it does work

chart_data = CategoryChartData()

In [6]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -12

# Create variables to be place into chart
debt_ebitda = msft.loc['Total Debt / EBITDA'].iloc[window:]
lev_limit = msft.loc['Leverage Limit'].iloc[window:]
equity_assets = msft.loc['Equity / Total Assets'].iloc[window:]

# Create dates
dates_str = msft.columns.astype(str)
dates_str = dates_str[window:]

# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('Total Debt / EBITDA', (debt_ebitda))
chart_data.add_series('Leverage Limit', lev_limit)
chart_data.add_series('Equity / Total Assets', equity_assets)

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 4: Composition of Operating Cash Flow**

In [7]:
# The code below shows the idx values for each placeholder
slide = prs.slides[3]

for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

0, TITLE (1)
1, OBJECT (7)
4, SLIDE_NUMBER (13)


In [8]:
# The code below is needed to access the chart on the third slide (i.e. prs.slides[2])
slide = prs.slides[3]
chart = slide.shapes[1].chart

chart_data = CategoryChartData()

In [9]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -7

# Create variables to be place into chart
net_income = msft.loc['Net Income'].iloc[window:]
dep_amort = msft.loc['Depreciation & Amort.'].iloc[window:]
non_cash = msft.loc['Other Non-Cash'].iloc[window:]
delta_wc = msft.loc['Change in Working Capital'].iloc[window:]
cfo = msft.loc['Cash from Operations'].iloc[window:]


# Create date
date = pd.Series(msft.columns[-7:])

In [10]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = date

# Update the actual data
chart_data.add_series('Net Income - slide', net_income)
chart_data.add_series('Depreciation & Amort.', dep_amort)
chart_data.add_series('Other Non-Cash', non_cash)
chart_data.add_series('Change in Operating Act.', delta_wc)
chart_data.add_series('Operating Cash Flow', cfo)

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 6: Table**

In [11]:
# The code below shows the idx values for each placeholder
slide = prs.slides[5]
for shape in slide.placeholders:
     print('%d, %s' % (shape.placeholder_format.idx, shape.name))

0, Title 2
1, Content Placeholder 4
4, Slide Number Placeholder 3


In [12]:
# Edit the sixth slide
slide = prs.slides[5]

# Update Slide Title
title_placeholder = slide.shapes.title
title_placeholder.text = 'Key Metrics' # the variable slide_title is defined above in the "Create Dynamic Text" section

# Format Slide Title
text_frame = title_placeholder.text_frame
p = text_frame.paragraphs[0]
run = p.add_run()
font = run.font
font.name = 'Arial'
font.size = Pt(18)
font.bold = False
font.color.rgb = RGBColor(109,109,109)

**Create data for table**

In [13]:
# This variable will determine how many periods of data are shown.  "-20" means select the last 20 rows of data. 
# In other words, select today minus 20 quarters. 

# NOTE: window does not have a negative before the 8 because the negative will mess up the for loops below
window = 8

# Create variables for chart data
ebitda = msft.loc['EBITDA',:][window:-2]
levered_fcf = msft.loc['Levered Free Cash Flow',:][window:-2]
leverage = msft.loc['Leverage',:][window:-2]

# Round variables to two digits
leverage = round(leverage,0)
levered_fcf = round(levered_fcf,0)



In [14]:
# Convert to strings because python_pptx tables can only handle strings
ebitda = [str(i) for i in ebitda]
levered_fcf = [str(i) for i in levered_fcf]
leverage = [str(i) for i in leverage]

# Add x to the end of every item in leverage
leverage = [leverage + 'x' for leverage in leverage]
#leverage

In [15]:
# Add comma after every third number
ebitda2 = []
for i in range(0, len(ebitda)):
    ebitda2.append("{:,}".format(float(ebitda[i])))

# Add comma after every third number
levered_fcf2 = []
for i in range(0, len(ebitda)):
    levered_fcf2.append("{:,}".format(float(ebitda[i])))

In [16]:
# Create dates
table_dates = msft.columns[-9:].tolist() # 9 instead of 8 because need to add the '$ Millions' in

In [17]:
# Remove the ".0" from each entry in ebitda
ebitda2 = [ebitda2[:-2] for ebitda2 in ebitda2]
# Remove the ".0" from each entry in leveraged_fcf
levered_fcf2 = [levered_fcf2[:-2] for levered_fcf2 in levered_fcf2]

In [ ]:
# Create table object - this is needed to add the data to the table
table = slide.shapes[2].table
table_data = CategoryChartData()

In [ ]:
for i in range(0, window):
        table.cell(0,i+1).text = table_dates[i] # start at column i + 1 to avoid writing over the row names
        
for i in range(0, window):
        table.cell(1,i+1).text = ebitda2[i]
        font.size = Pt(10)

for i in range(0, window):
        table.cell(2,i+1).text = levered_fcf2[i]
        font.size = Pt(10)

for i in range(0, window):
        table.cell(3,i+1).text = leverage[i]
        font.size = Pt(10)

# Save Presentation

In [ ]:
prs.save('General_Slides_OUTPUT.pptx')